In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

In [5]:
# tf.function and auto-graph.
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))


scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


In [6]:
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

10.3 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.78 ms ± 75.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# 1 + 1/2 + (1/2)^2 + ... + (1/2)^n

# @tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total
    
print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [8]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [9]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    is_positive = ag__.converted_call(tf.greater_equal, (z, 0.0), None, fscope)
    do_return = True
    retval_ = fscope.mark_return_value(scale * ag__.converted_call(tf.where, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, (z,), None, fscope)), None, fscope))
  do_return,
  return ag__.retval(retval_)

```

In [10]:
display_tf_code(converge_to_2)

```python
def tf__converge_to_2(n_iters):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('converge_to_2', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    total = ag__.converted_call(tf.constant, (0.0,), None, fscope)
    increment = ag__.converted_call(tf.constant, (1.0,), None, fscope)

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(iterates, increment, total):
      _ = iterates
      total += increment
      increment /= 2.0
      return increment, total
    increment, total = ag__.for_stmt(ag__.converted_call(range, (n_iters,), None, fscope), None, loop_body, get_state, set_state, (increment, total), ('increment', 'total'), ())
    do_return = True
    retval_ = fscope.mark_return_value(total)
  do_return,
  return ag__.retval(retval_)

```

In [11]:

var = tf.Variable(0.)

@tf.function
def add_21():
    return var.assign_add(21) # +=

print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [12]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

print(cube(tf.constant([1., 2., 3.])))
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [14]:
# 类型限制
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [17]:
# @tf.function py func -> graph
# get_concrete_function -> add input signature -> savedModel

cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

In [21]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1, 2, 3])))

True
True


In [22]:
cube_func_int32.graph

In [23]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [24]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [26]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [27]:
cube_func_int32.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [28]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [29]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 175
}